## Importação das bibliotecas

In [ ]:
import cv2
import numpy as np

## Leitura e configuração das imagens

In [ ]:
# Carrega a imagem de entrada
img = cv2.imread('jardim.jpg')

# Define a lista de objetos de interesse
objetos = ['vermelho.jpg', 'roxo.jpg]

# Converte a imagem de entrada para o espaço de cor HSV
hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

# Inicializa a imagem de backprojection
backproj = np.zeros_like(hsv_img[:,:,0]).astype(np.uint8)

## Backprojection

In [ ]:
for obj_name in objetos:
    # Carrega o objeto de interesse
    obj = cv2.imread(obj_name)

    # Converte o objeto de interesse para o espaço de cor HSV
    hsv_obj = cv2.cvtColor(obj, cv2.COLOR_BGR2HSV)

    # Calcula o histograma normalizado do objeto de interesse
    roi_hist = cv2.calcHist([hsv_obj], [0, 1], None, [180, 256], [0, 180, 0, 256])
    cv2.normalize(roi_hist, roi_hist, 0, 255, cv2.NORM_MINMAX)

    # Calcula o histogram backprojection da imagem de entrada
    backproj_obj = cv2.calcBackProject([hsv_img], [0, 1], roi_hist, [0, 180, 0, 256], 1)

    # Soma o backprojection do objeto de interesse à imagem de backprojection
    backproj += backproj_obj

## Remoção de ruídos e segmentação

In [ ]:
# Aplica filtro para remover ruídos
disc = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
cv2.filter2D(backproj, -1, disc, backproj)

# Aplica limiar para binarizar a imagem
ret,thresh = cv2.threshold(backproj, 50, 255, 0)
thresh = cv2.merge((thresh, thresh, thresh))

res = cv2.bitwise_and(img,thresh)
res = np.vstack((img, thresh, res))

## Salvar resultado

In [ ]:
cv2.imwrite('res.jpg', res)